In [1]:
import pandas as pd
import cv2
import os

In [2]:
mos_df = pd.read_csv("../data/MOS.csv") 
print(mos_df.head())

       image_id     mean      std
0  I01_01_1.bmp  5.51429  0.13013
1  i01_01_2.bmp  5.56757  0.16008
2  i01_01_3.bmp  4.94444  0.15186
3  i01_01_4.bmp  4.37838  0.11046
4  i01_01_5.bmp  3.86486  0.10229


---
#### imread stores image as a 3D array [height, width, channels]
#### cvtColor converts the color space from BGR to RBG 
---

In [3]:
img_path = os.path.join("../data/distorted_images", mos_df['image_id'][0])
img = cv2.imread(img_path)
print(img_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

../data/distorted_images\I01_01_1.bmp


---

#### Takes in an image as a parameter 
#### Converts it to grayscale 
#### Laplacian takes the second derivative of pixel intensities and then we calculate variance to decide if the image is sharp 
#### Brightness is determined by the average pixel intensity of the grayscale image 
#### Noise is determined by the average standard deviation of the grayscale image 
#### color_var = np.std(img, axis=(0,1)).mean() calculates the standard deviation across each color channel and then takes the mean to see if it is monochrome or saturated 

In [4]:
import numpy as np

def extract_features(img):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sharpness = cv2.Laplacian(gray, cv2.CV_64F).var()
    brightness = np.mean(gray)
    noise = np.std(gray)
    color_var = np.std(img, axis=(0,1)).mean()
    return [sharpness, brightness, noise, color_var]


--- 
#### Features store all the feature data and labels store the mean opinion score for the image 
#### This function loads each image in the distorted data folder, converts it to RGB, extracts its features, and appends the mean optimised score for the image to the row

In [5]:
features = []
labels = []

for idx, row in mos_df.iterrows():
    img_path = os.path.join("../data/distorted_images", row['image_id'])
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    features.append(extract_features(img))
    labels.append(row['mean'])

import pandas as pd
features_df = pd.DataFrame(features, columns=['sharpness','brightness','noise','color_var'])
features_df['mean'] = labels


In [6]:
features_df.describe()

,sharpness,brightness,noise,color_var,mean
count,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000
mean,3090.827752,110.978255,48.926543,50.499069,4.474657
std,5174.869286,25.223155,12.731646,12.810518,1.239880
min,1.146698,37.158381,12.874847,13.868108,0.242420
25%,673.165757,91.945970,40.082295,42.334497,3.611110
50%,1578.196417,107.922424,47.560431,49.049925,4.600000
75%,3014.812871,123.465853,53.874086,55.619075,5.526320
max,39899.549265,192.376226,103.901088,105.958672,7.214290


In [14]:
param_dist = {
    'n_estimators': [100, 200, 300, 400, 500],       # number of trees
    'max_depth': [None, 10, 20, 30, 40],            # maximum depth
    'min_samples_split': [2, 5, 10, 15],            # min samples to split a node
    'min_samples_leaf': [1, 2, 4, 6],               # min samples at leaf node
    'max_features': ['sqrt', 'log2', None]          # number of features at each split
}

In [15]:
from sklearn.model_selection import RandomizedSearchCV
rf = RandomForestRegressor(random_state=42)

random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=50,                # test 50 random combinations
    cv=5,                     # 5-fold cross-validation
    scoring='neg_mean_squared_error',
    n_jobs=-1,                # use all CPU cores
    verbose=2,
    random_state=42
)

In [16]:
random_search.fit(X_train, y_train)

# Best hyperparameters
print("Best parameters:", random_search.best_params_)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best parameters: {'n_estimators': 300, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 30}


In [18]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

X = features_df.drop(columns=['mean'])
y = features_df['mean']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(
    n_estimators=300,
    min_samples_split=5,
    min_samples_leaf=1,
    max_features='sqrt', 
    max_depth=30,
    random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("Test MSE:", mse)


Test MSE: 0.9186281973897731


In [9]:
import numpy as np

# Baseline MSE: predict the mean MOS for all images
baseline_mse = np.mean((features_df['mean'] - features_df['mean'].mean())**2)
baseline_rmse = np.sqrt(baseline_mse)

print("Baseline MSE:", baseline_mse)
print("Baseline RMSE:", baseline_rmse)

Baseline MSE: 1.5367897131527357
Baseline RMSE: 1.2396732283762264
